# Creating a Birthchart Column for Each DataFrame

Here, we will be using some kind of massive for-loop to get the birthchart info NOT ONLY for each celebrity in an occupation dataframe, BUT ALSO the birthchart info for each dataframe -- in effect, we will be getting everyone's birthchart info on a dataframe somewhere.

We will be using converting-messy-info-clumn-into-birthtime-and-birthplace-take2 and converting-messy-info-clumn-into-birthtime-and-birthplace-take1 as models to generalize.

With that in mind, I'm gonna take some writen notese on converting-messy-info-clumn-into-birthtime-and-birthplace-take2, outlining what happens at every step -- that way I can build a cleaner function here.

In [16]:
#Imports
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re
import string

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.chart import Chart
from flatlib import const

from timezonefinder import TimezoneFinder
from datetime import datetime
from pytz import timezone, utc


Last time, I just straight up read a particular csv at this point. But now, since I'm gonna be dealing with multiple csvs, I need to begin my for loop -- then I can read each csv individually by name. To get this to work properly, I need to get that old occupations_list_shortened I had before. I'm just gonna copy and paste it in here.

In [17]:
occupations_list_shortened = [
'r_scientists-inventors',
'r_chemists',
'r_cooks',
'r_doctors',
'r_physicists',
'r_entrepreneurs',
'r_designers',
'r_architects',
'r_fashion-models',
'r_porn-stars',
'r_journalists-publicists',
'r_poets',
'r_writers',
'r_artists',
'r_painters',
'r_photographers',
'r_sculptors',
'r_rulers',
'r_presidents',
'r_priests-theologians',
'r_politicians',
'r_founders',
'r_mathematicians',
'r_philosophers',
'r_basketball-players',
'r_baseball-players',
'r_martial-artists',
'r_racers',
'r_tennis-players',
'r_actors',
'r_producers',
'r_script-writers',
'r_dancers',
'r_directors-ceos',
'r_singers',
'r_musicians',
'r_composers',
'r_lawyers',
'r_pilots-aviators',
'r_soldiers',
'r_activists',
'r_presenters']

At this point, I really only have the data of the occupations up until the tennis-players one (inclusive). So I'll may an occupations_list_shortened_var which is just a partial version of the occupations_list_shortened that I can change when it's convenient.

In [18]:
occupations_list_shortened_var = occupations_list_shortened[:29]
#print(occupations_list_shortened_var)

Now we loop over these occupations.

And before that, we have to put in all our methods in such a way that they are df-flexible.

In [21]:
def adios_weird_characters(i):
    s = df.iloc[i].info
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, s))

def get_birthtime_info(row):
    b_info = df['info'][row]
    if '*' in b_info:
        return None
    if '(gregor.)' in b_info:
        #b_info_mod_time = [b_info_mod_time[0][:-9]]
        b_info = b_info.replace('(gregor.)','')
    elif 'h(gregor.' in b_info:
        #b_info_mod_time = b_info_mod_time[:-8]
        b_info = b_info.replace('(gregor.','')
    b_info_mod = ''
    if 'Gender:FemaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:FemaleDate of Birth:', '')
    elif 'Gender:MaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
    if 'Birth place:' in b_info_mod:
        b_info_mod = b_info_mod.split('Birth place:')
    elif 'Country:' in b_info_mod:
        b_info_mod = b_info_mod.split('Country:')
    b_info_mod_time = b_info_mod[0]
    
    b_info_mod_time = b_info_mod_time.split(' - ')
    #print(b_info_mod_time)
    #print('...end of get_birthtime_info')
    
    return b_info_mod_time

def get_birthplace_info(row):
    b_info = df['info'][row]
    if '*' in b_info:
        return None
    if '(gregor.)' in b_info:
        #b_info_mod_place = b_info_mod_place[:-9]
        b_info = b_info.replace('(gregor.)','')
    elif '(gregor.' in b_info:
        #b_info_mod_place = b_info_mod_place[:-8]
        b_info = b_info.replace('h(gregor.','')
    b_info_mod = ''
    if 'Gender:FemaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:FemaleDate of Birth:', '')
    elif 'Gender:MaleDate of Birth:' in b_info:
        b_info_mod = b_info.replace('Gender:MaleDate of Birth:', '')
    else:
        return None
    if 'Birth place:' in b_info_mod:
        b_info_mod = b_info_mod.split('Birth place:')
    elif 'Country:' in b_info_mod:
        b_info_mod = b_info_mod.split('Country:')
    b_info_mod_place = b_info_mod[1].split('Age:')
    b_info_mod_place = b_info_mod_place[0]
    b_info_mod_place = b_info_mod_place.split('Country:')
    #print(b_info_mod_place)
    #print('...end of get_birthplace_info')
    
    return b_info_mod_place

def convert_month_name_to_number(month):
    month = month.lower()
    if month == 'january':
        return 1
    elif month == 'february':
        return 1
    elif month == 'march':
        return 1
    elif month == 'april':
        return 1
    elif month == 'may':
        return 1
    elif month == 'june':
        return 1
    elif month == 'july':
        return 1
    elif month == 'august':
        return 1
    elif month == 'september':
        return 1
    elif month == 'october':
        return 1
    elif month == 'november':
        return 1
    elif month == 'december':
        return 1
    else:
        #print('Not_a_month')
        return None

def time_formatter(time_lst):
    if time_lst == None:
        return None
    if len(time_lst) == 2:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        time = time_lst[1]
        time = time[:-1]
        ans = [year, convert_month_name_to_number(month), day, time]
        #print(ans)
        return ans
    elif len(time_lst) == 1:
        first = time_lst[0]
        month = " ".join(re.split("[^a-zA-Z]*", first))
        month = month.replace(' ', '')
        first = first.replace(' ', '')
        temp = first.split(month)
        day = temp[0]
        year = temp[1]
        ans = [year, convert_month_name_to_number(month), day]
        #print(ans)
        return ans
    return 'No_time_info'
        
    
def place_formatter(place_lst):
    if place_lst == None:
        return None
    if len(place_lst) == 1:
        name = place_lst[0].split(', ')
        ans = name[1]
        return ans
    else:
        city_info = place_lst[0]
        country_info = place_lst[1]
        if ',' in city_info:
            city_info = city_info.split(', ')
        else:
            city_info = [city_info]
        if ',' in country_info:
            country_info = country_info.split(', ')
            country_info = country_info[1]
        if len(city_info) == 2:
            city = city_info[0]
            state = city_info[1]
        elif len(city_info) == 1:
            city = city_info[0]
            state = 'no_state'
        country = country_info
        if state == 'no_state':
            ans = [city, country]
            ans = ', '.join(ans)
        else:
            ans = [city, state, country]
            ans = ', '.join(ans)
        #print(ans)
        return ans

def make_time_column(row):
    return time_formatter(get_birthtime_info(row))

def make_place_column(row):
    return place_formatter(get_birthplace_info(row))

def point_formatter(row):
    p = df.iloc[row].point
    if p == None:
        return None
    p_ns = p[0]
    p_ew = p[1]
    if p_ns > 0:
        p_ns_mod = str(round(p_ns, 4)).replace('.', 'n')
    else:
        p_ns_mod = str(round(-p_ns, 4)).replace('.', 's')
    if p_ew > 0:
        p_ew_mod = str(round(p_ew, 4)).replace('.', 'e')
    else:
        p_ew_mod = str(round(-p_ew, 4)).replace('.', 'w')
    return (p_ns_mod,p_ew_mod)

def get_offset(lat, lng):
    tf = TimezoneFinder()

    today = datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lng=lng, lat=lat))
    today_target = tz_target.localize(today)
    today_utc = utc.localize(today)
    val = (today_utc - today_target).total_seconds()/60
    hr = int(val // 60)
    hr = str(hr).zfill(2)
    mn = int(val - 60 * int(hr))
    mn = str(mn).zfill(2)
    ans = hr + ':' + mn
    return ans

def date_formatter(row):
    old_date = df.iloc[row].birthtime
    
    year = str(old_date[0]).zfill(4)
    month = str(old_date[1]).zfill(2)
    day = str(old_date[2]).zfill(2)
    new_date = str(year) + '/' + str(month) + '/' + str(day)
    
    location = df.iloc[row].point
    lat = location[0]
    lng = location[1]
    if 'n' in lat:
        lat = lat.replace('n', '.')
    if 's' in lat:
        lat = '-' + (lat.replace('s', '.'))
    if 'e' in lng:
        lng = lng.replace('e', '.')
    if 'w' in lng:
        lng = '-' + (lng.replace('w', '.'))
    offset = get_offset(float(lat), float(lng))
    
    if len(old_date) == 4:
        time = str(old_date[3])
        ans = Datetime(new_date, time, offset)
    else:
        ans = Datetime(new_date, offset)
    return ans

def chart_creator(row):
    if df.iloc[row].point == None or df.iloc[row].location == None or df.iloc[row].birthplace == None or df.iloc[row].birthtime == None:
        return None
    d = date_formatter(row)
    #^Remember to use the same df here.
    l = df.iloc[row].point
    pos = GeoPos(l[0], l[1])
    try:
        chart = Chart(d, pos)
        return chart
    except:
        None

In [24]:
#occupations_list_shortened_var_temp = [occupations_list_shortened[3]]

for o in occupations_list_shortened_var:
#for o in occupations_list_shortened_var_temp:
    print(o)
    o_name = o[2:]
    o_csv_str = 'hi_its_' + o_name + '_df.csv'
    df = pd.read_csv(o_csv_str)
    
    df['count'] = np.arange(len(df))
    df['info_norm'] = df['count'].apply(adios_weird_characters)
    df = df.drop(['info'], axis=1)
    df = df.rename(columns={"info_norm": "info"})
    
    df['birthtime'] = df['count'].apply(make_time_column)
    df['birthplace'] = df['count'].apply(make_place_column)
    
    df = df.drop(['info'], axis=1)
    
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    df['location'] = df['birthplace'].apply(geocode)
    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
    
    df['point'] = df['count'].apply(point_formatter)
    
    df['chart'] = df['count'].apply(chart_creator)
    
    df = df[df['chart'].map(lambda x: x!=None)]
    
    o_new_csv_str = 'r_' + o_name + '_charted.csv'
    df.to_csv(o_new_csv_str, index=False)
    #print(df)



r_scientists-inventors


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Belfast, Northern Ireland, United Kingdom',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/li

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Mnchengladbach, Germany',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/c

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Milwaukee, Wisconsin, United States',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/pyth

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Schweinfurt, Germany',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/clie

...

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Strasbourg, France',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/client

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Besanon, France',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3.7/http/client.py

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Duluth, Minnesota, United States',), **{}).
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/lib/python3.7/http/client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/lib/python3.7/http/client.py", line 1016, in _send_output
    self.send(msg)
  File "/anaconda3/lib/python3.7/http/client.py", line 956, in send
    self.connect()
  File "/anaconda3/lib/python3.7/http/client.py", line 1384, in connect
    super().connect()
  File "/anaconda3/lib/python3

r_cooks


IndexError: string index out of range